In [3]:
import pandas as pd
import numpy as np
import string
import re
from collections import Counter 
import nltk 
from nltk.tokenize import word_tokenize,sent_tokenize                                          
from nltk.corpus import stopwords
import gensim
from gensim import corpora
import pickle
from nltk.corpus import wordnet
import spacy
from fuzzywuzzy import fuzz

/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [88]:
df_reviews=pd.read_csv('Text_Review_Dump_July.csv', encoding='latin', low_memory=False)

In [89]:
df_product=pd.read_csv('FlipKart.csv', encoding='latin', low_memory=False)

In [90]:
df=pd.merge(left=df_reviews, right=df_product, left_on='productid', right_on='pid', how='left', indicator=True)

In [91]:
# df = pd.read_csv('bags_data.csv', encoding='latin', low_memory=False)

In [92]:
list_cat=['Home Appliances','Wrist Watches','Headphones', 'Mobiles', "Men's Clothing",
          "Women's Clothing",'Makeup','Tablet Accessories','Tablets',"Men's Footwear","Body & Face Skin Care","Kids' Clothing",
          "Mobile Accessories","Bags & Backpacks","Hair Care and Accessory","Personal Care Appliances","Televisions",
          "Fragrances","Kitchen Appliances","Women's Footwear","Storage",
          "Speakers","Bed Linen & Blankets","Handbags & Clutches",
          "Health & Nutrition","Smart Bands","Wallets & Clutches",
          "Health Care","Utility Lighting","Luggage & Travel",
          "Laptops","Laptop Accessories","Cameras","Camera Accessories",
          "Shaving & Beard Care","Smart Watches","Gaming Accessories","Gaming Consoles"]

In [93]:
len(list_cat)
df.rename(columns = {'Cat1':'Category 1','Cat2':'Category 2','Cat3':'Category 3','Cat4':'Category 4'}, inplace=True)
df.head()

,ï»¿,producttitle,reviewtitle,reviewbody,rating,createddate,referenceid,productid,flu_id,version,...,pid,Breadcrumbs,Category 1,Category 2,Category 3,Category 4,Cat5,Cat6,Cat7,_merge
0,0,Samsung 7.2 kg Semi Automatic Top Load Washing...,Best in the market!,simple and elegant.. nice design and colors.. ...,5,2019-07-01 00:00:04,e1995b61-921c-41be-8a21-f6bca614deea,WMNFDV6GWSMNGFGC,0fe98451-441e-4d4e-b833-c4f655368c0f,1,...,WMNFDV6GWSMNGFGC,Home,Home & Kitchen,Home Appliances,Washing Machines,Samsung Washing Machines,NaN,NaN,NaN,both
1,1,Beardo Beard Growth Hair Oil,Slightly disappointed,Note that much effective ....and help only tho...,2,2019-07-01 00:00:22,f389557a-27c3-4f78-ab96-c73012e2629b,HOLEEGFQCGDFEJSF,672b577d-ed3e-4b1b-b609-5ed04eceb726,1,...,HOLEEGFQCGDFEJSF,Home,Beauty and Grooming,Shaving & Beard Care,Beard Oil,Beardo Beard Oil,NaN,NaN,NaN,both
2,2,Fastrack Rectangular Sunglasses,Very Good,mst,4,2019-07-01 00:01:36,af5145d2-606b-4def-addf-6e4cc54aa0e1,SGLDNGSBQGZBGHG3,8e640e1a-3f3c-40ce-994d-580ca95fe178,1,...,SGLDNGSBQGZBGHG3,Home,Sunglasses,Fastrack Sunglasses,NaN,NaN,NaN,NaN,NaN,both
3,3,Pigeon FAVOURITE Electric Kettle,Super!,Ek dam badiya verybfast working,5,2019-07-01 00:00:25,797f149d-9927-4908-89dc-cd2558492cab,EKTEASGCGV6GYP6S,a89d25f6-a379-44b3-8e4a-fb59dbe4217b,1,...,EKTEASGCGV6GYP6S,Home,Home & Kitchen,Kitchen Appliances,Electric Jug(heater)/Travel Kettles,Pigeon Electric Jug(heater)/Travel Kettles,NaN,NaN,NaN,both
4,4,Huppme Happy Birthday Siya name coffee mug Cer...,Terrific,hhhhj,5,2019-07-01 00:00:40,d88eeb83-7117-4da5-b7a3-f8e5613e134f,MUGEZHG3EEHG6GMN,7cba31cc-8bad-4437-a083-aa50c401b6c0,1,...,MUGEZHG3EEHG6GMN,Home,Kitchen,NaN,NaN,NaN,NaN,NaN,NaN,both


In [210]:
# df_=df[df['Category 2'].isin(["Kitchen Appliances","Personal Care Appliances", "Home Appliances"])]
# print(set(df['Category 2']))
df_=df[df['Category 2'].isin(["Personal Care Appliances"])]

In [211]:
print(len(df_))
from re import search
df_=df_[df_['Category 3'].str.startswith("Shavers", na=False)]
print(set(df_['Category 3']))

13915
{'Shavers'}


In [212]:
# list_sub_cat=['Ethnic Wear',
#  'Western Wear',
#  'Lingerie, Sleep & Swimwear',
#  'Winter & Seasonal Wear',
#  'Sports & Gym Wear',
#  'Fusion Wear',
#  'Accessories',
#  'Maternity Wear',
#  'Formal Wear']
# df_women=df_[df_['Category 2']=="Women's Clothing"]
# df_women

In [213]:
# df_women=df_women[df_women['Category 3'].isin(list_sub_cat)]

In [214]:
# len(df_women)

In [215]:
# list_sub_cat=[
#  'T-Shirts',
#  'Inner Wear & Sleep Wear',
#  'Shirts',
#  'Winter & Seasonal Wear',
#  'Sports Wear',
#  'Jeans',
#  'Accessories & Combo Sets',
#  'Cargos, Shorts & 3/4ths',
#  'Trousers',
#  'Ethnic Wear',
#  'Suits & Blazers',
#  'Fabrics']

In [216]:
# #men's
# df_men=df_[df_['Category 2']=="Men's Clothing"]
# df_men
# df_men=df_men[df_men['Category 3'].isin(list_sub_cat)]
# len(df_men)

In [217]:
# list_sub_cat=[
#  'Girls Wear',
#  'Boys Wear',
#  'Infants Wear']

In [218]:
# df_kids=df_[df_['Category 2']=="Kids' Clothing"]
# df_kids=df_kids[df_kids['Category 3'].isin(list_sub_cat)]
# len(df_kids)

In [219]:
# df_clothing=pd.DataFrame()

In [220]:
# df_clothing=df_clothing.append(df_women,ignore_index=True)
# len(df_clothing)

In [221]:
# df_clothing=df_clothing.append(df_men)
# len(df_clothing)

In [222]:
# df_clothing=df_clothing.append(df_kids)

In [223]:
# len(df_clothing)

In [224]:
# df_new=df_clothing[['producttitle', 'reviewtitle', 'reviewbody', 'rating','referenceid','Category 1',
#        'Category 2', 'Category 3']]
df_new=df_[['producttitle', 'reviewtitle', 'reviewbody', 'rating','referenceid','Category 1',
       'Category 2', 'Category 3']]

In [225]:
df_new.drop_duplicates( subset=['referenceid'],inplace=True)

/Users/harsha/Downloads/Programs/python_venv3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [226]:
df_new=df_new.reset_index()
len(df_new)

147

In [227]:
# df_new['reviewbody'].isnull()

In [228]:
df_new["review_sentence"] = df_new["reviewtitle"].map(str)+" . " + df_new["reviewbody"]
(df_new["review_sentence"])

0      Very Good . Nice removes hair very quickly.......
1      Bad quality . this is a 'elephant in the house...
2      Hated it! . Making fool of customer, sent diff...
3       Worth the money . Good product but packaging old
4                                    Nice product . ????
                             ...                        
142                  Hated it! . I received used product
143    Did not meet expectations . This product is ha...
144                Did not meet expectations . an useful
145                                   Fabulous! . ??????
146    Slightly disappointed . balo k size k liye jo ...
Name: review_sentence, Length: 147, dtype: object

In [229]:
def prepare_sentences(sentence):
    sentence=str(sentence)
    sentence=sentence.lower().strip()
    sentence=sent_tokenize(sentence)
    sentence=[re.sub(r'\.+', ".", sent) for sent in sentence]
#     sentence=[sent.translate(str.maketrans('', '', string.punctuation)) for sent in sentence]
#     print(sentence)
    sentence=[sent.translate(str.maketrans('', '', string.digits)) for sent in sentence]
    sentence = [re.sub('\S*@\S*\s?', '', sent) for sent in sentence]#removing emails 
#     print(sentence)
    sentence = [re.sub('â€œ', '', sent) for sent in sentence]
#     sentence = [re.sub('\x95', '', sent) for sent in sentence]
#     sentence = [re.sub('\x92', '', sent) for sent in sentence]
    sentence = [re.sub(r'[^\x00-\x7f]',r'', sent)  for sent in sentence]
    sentence = [re.sub('!', '', sent) for sent in sentence]
#     sentence = [re.sub('?', '.', sent) for sent in sentence]
    sentence = [re.sub(':', '', sent) for sent in sentence]
    sentence = [re.sub('\s+', ' ', sent) for sent in sentence]  #removing newline chars like \n etc
    sentence = [re.sub("\''", "", sent) for sent in sentence] 
    sentence=[sent.translate(str.maketrans('', '', string.punctuation)) for sent in sentence]
#     print(sentence)
    sentence=[sent for sent in sentence if len(sent)!=0]
    print(sentence)
    return sentence

In [230]:
def unnest(df, tile, explode):
    vals = df[explode].sum(1)
    rs = [len(r) for r in vals]
    a = np.repeat(df[tile].values, rs, axis=0)
    b = np.concatenate(vals.values)
    d = np.column_stack((a, b))
    return pd.DataFrame(d, columns = tile +  ['_'.join(explode)])

In [231]:
df_new['sentences']=df_new['review_sentence'].apply(lambda x :prepare_sentences(x))
print(df_new["sentences"].head())
df_new.dropna(inplace=True)
data=unnest(df_new, ['producttitle', 'reviewtitle', 'reviewbody', 'rating','referenceid','Category 1',
       'Category 2', 'Category 3'], ['sentences'])

len(data)
data.dropna(subset=['sentences'],inplace=True)
# print(data.head())

['very good ', 'nice removes hair very quicklywe just have to rotate in a circular motion', 'there was a block for battery but they havent provided it']
['bad quality ', 'this is a elephant in the house product']
['hated it', 'making fool of customer sent different product as they have shown']
['worth the money ', 'good product but packaging old']
['nice product ']
['unsatisfactory ']
['dont waste your money ', 'pura dhatiya product purchase kiye huve mahine me blade bharab ho gaya pura paisa pani me gaya']
['good choice ', 'very good product ']
['not recommended at all ', 'the machine appeared to be used', 'the pack does not have the battery as shown in figure in pack insert', 'a temporary battery is inserted into the machine', 'i had heard about such fraud from flip kart but this incident proved it right', 'photos attached', 'system not allowing it to be returned', 'disappointed', 'will not use flipkart anymore']
['terrible product ', 'flipkart not send as same product its different 

In [232]:
stop_words = stopwords.words('english')
extra_stop_words=['could','would','k','time','hour','minute','year','month','love','day','bit','good','thank','job','thanks','rubbish']
stop_words.extend(extra_stop_words)
def sent_to_words(sentences):
    words = set(nltk.corpus.words.words())
    word_list=[]
    for sentence in sentences:
        word_list.append([i for i in word_tokenize(sentence) if i not in stop_words  ]) #and and (i.lower() in words or not i.isalpha())
    return word_list

In [233]:
# stop_words = stopwords.words('english')
# extra_stop_words=['could','would','k','time','hour','minute','year','month','day','today']
# stop_words.extend(extra_stop_words)
# def sent_to_words(sentences):
#     words = set(nltk.corpus.words.words())
#     word_list=[]
#     for sentence in sentences:
#         word_list.append([i for i in word_tokenize(sentence) if i not in stop_words and (i.lower() in words or not i.isalpha())]) #and i.lower() in words or not i.isalpha()
#     return word_list
# print(data.head())

In [234]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
#         print("Token",[token.lemma_ for token in doc])
#         texts_out.append([token.lemma_ for token in doc])
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [235]:
data_for_lemma = data.sentences.values.tolist()
data_sent=data_for_lemma
# print(data.sentences.values.tolist()[1:10])

In [236]:
data_for_lemma=[x for x in data_for_lemma if x!=" " or x!=""]
print(len(data_for_lemma))
data_for_lemma=[ x.strip() for x in data_for_lemma ]
#data_for_lemma

357


In [237]:
data_words = (sent_to_words(data_for_lemma))
data_words = [x for x in data_words if x]
# data_words
# print(data_words[1:100])

In [238]:
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN'])    #can modify parts of speech as required here
# print('lemmatized data: ',data_lemmatized)

In [239]:
data_lda=data_lemmatized #=[y for x in data_lemmatized for y in x if y
data_lemmatized=[y for x in data_lemmatized for y in x if y and y not in stop_words]

In [164]:
# same_keys=[]
# for i in data_lemmatized:
#     x=[j for j in data_lemmatized if (fuzz.ratio(j,i)>80) ]
#     x=(set(x))
#     print(x)
        

In [165]:
# same_keys

In [1274]:
#remove all words which are stopwords and not an alphabet
words = set(nltk.corpus.words.words())
data_lemmatized_new=[ i for i in data_lemmatized if i not in stop_words and (i.lower() in words or not i.isalpha())] 

In [90]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import defaultdict
#print("processed_data",data_lemmatized)
vectorizer=TfidfVectorizer(max_features=100)
processed_data=vectorizer.fit_transform(data_lemmatized_new)
# indices = np.argsort(vectorizer.idf_)[::-1]
feature_names = vectorizer.get_feature_names()
print("Feature names :",feature_names)


Feature names : ['aluminum', 'baby', 'bag', 'body', 'boy', 'brand', 'buy', 'camera', 'capability', 'cause', 'choice', 'class', 'company', 'compare', 'cost', 'damage', 'delivery', 'emphasis', 'expectation', 'experience', 'finishing', 'fit', 'foot', 'grip', 'hand', 'handle', 'head', 'height', 'hight', 'hold', 'holder', 'home', 'image', 'inch', 'issue', 'item', 'kit', 'leg', 'lens', 'level', 'light', 'lock', 'market', 'material', 'metal', 'mobile', 'money', 'night', 'one', 'option', 'order', 'package', 'part', 'party', 'penny', 'phone', 'photo', 'picture', 'piece', 'place', 'plastic', 'point', 'price', 'problem', 'product', 'professional', 'purchase', 'quality', 'range', 'rate', 'refund', 'replace', 'replacement', 'return', 'review', 'rotating', 'satisfying', 'screw', 'seller', 'service', 'shoot', 'shopping', 'side', 'size', 'spirit', 'stability', 'stand', 'star', 'strength', 'thickness', 'thing', 'type', 'use', 'user', 'value', 'video', 'waste', 'week', 'weight', 'work']


In [91]:
from collections import Counter
# Pass the split_it list to instance of Counter class. 
Counter = Counter(data_lemmatized_new) 
  
# most_common() produces k frequently encountered 
# input values and their respective counts. 
most_occur = Counter.most_common(150)
  
# print(most_occur)
most_occuring=[word for word,freq in most_occur ]
print(most_occuring)

['product', 'quality', 'money', 'price', 'purchase', 'delivery', 'phone', 'experience', 'expectation', 'camera', 'market', 'leg', 'holder', 'damage', 'penny', 'use', 'video', 'height', 'item', 'weight', 'seller', 'mobile', 'material', 'return', 'size', 'replacement', 'choice', 'level', 'piece', 'baby', 'range', 'hold', 'value', 'service', 'waste', 'class', 'boy', 'order', 'part', 'thing', 'stand', 'point', 'grip', 'star', 'photo', 'cost', 'light', 'option', 'aluminum', 'picture', 'cause', 'night', 'thickness', 'foot', 'problem', 'plastic', 'brand', 'inch', 'strength', 'satisfying', 'package', 'screw', 'head', 'party', 'bag', 'body', 'review', 'handle', 'replace', 'shoot', 'hand', 'finishing', 'stability', 'work', 'buy', 'rotating', 'kit', 'lens', 'spirit', 'lock', 'issue', 'image', 'side', 'week', 'metal', 'user', 'request', 'treasure', 'tripod', 'rate', 'wind', 'mechanism', 'compare', 'home', 'place', 'h', 'box', 'budget', 'emphasis', 'date', 'refund', 'fit', 'shopping', 'company', 'h

In [92]:
category={
    
    "0":['money','buy','price', 'offer', 'discount','sale', 'budget','purchase','penny','payment','pricing','cost', 'range', 'mrp','rs', 'rupees'],
    "1":['deliver', 'courier',"delivery",'guy'],
    "2":['customer',"service",'flipkart','company','user','consumer','warranty','experience'],
    "3":['replacement','refund','cancel','exchange','replace','policy','return'],
    "4":['package','shipment','packaging','packaged','box','packing'],
    "5":['buyer','seller','distributor','market'],
    "6":['fake','fraud'],
    "7":['response','week','complaint','action','today','mail']
}

In [93]:
list_avoid=[i for x in category.values() for i in x]

In [94]:
# list_avoid

In [95]:
ac=[]
# most_occuring.extend(feature_names)
for i in most_occuring:
    if i not in (list_avoid):
        ac.append(i)
print(ac)
# print(len(ac))
# print((set(feature_names) - set(most_occuring)))

['product', 'quality', 'phone', 'expectation', 'camera', 'leg', 'holder', 'damage', 'use', 'video', 'height', 'item', 'weight', 'mobile', 'material', 'size', 'choice', 'level', 'piece', 'baby', 'hold', 'value', 'waste', 'class', 'boy', 'order', 'part', 'thing', 'stand', 'point', 'grip', 'star', 'photo', 'light', 'option', 'aluminum', 'picture', 'cause', 'night', 'thickness', 'foot', 'problem', 'plastic', 'brand', 'inch', 'strength', 'satisfying', 'screw', 'head', 'party', 'bag', 'body', 'review', 'handle', 'shoot', 'hand', 'finishing', 'stability', 'work', 'rotating', 'kit', 'lens', 'spirit', 'lock', 'issue', 'image', 'side', 'metal', 'request', 'treasure', 'tripod', 'rate', 'wind', 'mechanism', 'compare', 'home', 'place', 'h', 'emphasis', 'date', 'fit', 'shopping', 'hight', 'u', 'one', 'future', 'capability', 'type', 'take', 'mat', 'professional', 'rod', 'electronics', 'watch', 'buck', 'alloy', 'carry', 'recording', 'length', 'friend', 'number', 'purpose', 'beginner', 'card', 'parcel'

In [51]:
# import numpy as np
# import sklearn.cluster
# import distance

# ac = np.array(ac)
# lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in ac] for w2 in ac])

# affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping=0.5)
# affprop.fit(lev_similarity)
# for cluster_id in np.unique(affprop.labels_):
#     exemplar = ac[affprop.cluster_centers_indices_[cluster_id]]
#     cluster = np.unique(ac[np.nonzero(affprop.labels_==cluster_id)])
#     cluster_str = ", ".join(cluster)
#     print(" - *%s:* %s" % (exemplar, cluster_str))

In [52]:
# from sklearn.cluster import MiniBatchKMeans
# from sklearn.decomposition import PCA
# import matplotlib.pyplot as plt

# random_state = 0 

# from sklearn.cluster import KMeans

# vectorizer=TfidfVectorizer(max_features=100)
# processed_data=vectorizer.fit_transform(data_lemmatized_new)
# feature_names = vectorizer.get_feature_names()
# print("Feature names :",feature_names)
# # vec.fit(data_lemmatized_new)
# # features = vec.transform(data_lemmatized_new)

# # vec.fit(feature_names)
# # features = vec.transform(feature_names)


# # cls = MiniBatchKMeans(n_clusters=4, random_state=random_state)
# # cls.fit(features)

# # # predict cluster labels for new dataset
# # cls.predict(features)

# # # to get cluster labels for the dataset used while
# # # training the model (used for models that does not
# # # support prediction on new dataset).
# # cls.labels_
# # print((cls.labels_))

# true_fit = 5
# model = KMeans(n_clusters=true_fit, init='k-means++', max_iter=100, n_init=1)
# model.fit(processed_data)

# print("Top terms per cluster:")
# order_centroids = model.cluster_centers_.argsort()[:, ::-1]
# terms = vectorizer.get_feature_names()
# for i in range(true_fit):
#     print("Cluster %d:" % i),
#     for ind in order_centroids[i, :10]:
#         print(' %s' % terms[ind]),
#     print
# lines_for_predicting = ["Perfect product! . superb wax i trying is product", "fragarnce of the deo is very good"]
# kmeans.predict(vectorizer.transform(lines_for_predicting))

In [55]:
# from gensim.models.keyedvectors import KeyedVectors

# glove_model = KeyedVectors.load_word2vec_format("glove.6B.200d.w2vformat.txt", binary=False)

In [96]:
# ac.remove('pantie')
# ac.remove('prefilter')
for a in ac:
    closest=glove_model.most_similar(a,topn=50)
    closest=[n for n,g in closest]
    x=[i for i in ac if i in closest]
    print(a,x)

product ['quality', 'value', 'brand']
quality ['product', 'level', 'value', 'work']
phone ['mobile']
expectation ['value', 'doubt']
camera ['video', 'photo', 'picture', 'lens', 'image', 'watch']
leg ['foot', 'side']
holder ['record']
damage ['cause']
use ['take']
video ['camera', 'picture', 'recording']
height ['weight', 'size', 'level', 'thickness', 'length', 'limit']
item ['product', 'choice', 'piece', 'value', 'issue', 'date', 'store']
weight ['height', 'size', 'strength', 'body', 'length', 'limit']
mobile ['phone']
material ['use', 'piece', 'metal', 'cover']
size ['height', 'weight', 'length']
choice ['option', 'one']
level ['quality', 'rate']
piece ['material', 'part', 'thing', 'hand', 'work', 'one']
baby ['boy']
hold ['take']
value ['product', 'quality']
waste []
class ['level', 'part', 'one', 'type', 'b']
boy ['baby', 'friend']
order ['take']
part ['one']
thing []
stand ['hold', 'take']
point ['one', 'take']
grip ['hold', 'hand', 'tight']
star ['one']
photo ['camera', 'video', '

In [97]:
# for a in ac:
#     word_list = []
#     for b in ac:
#         closest = glove_model.most_similar_cosmul(positive =[a,b], topn= 10)
#         closest = [word for word,prob in closest]
#         list = [i for i in ac if i in closest and i not in word_list]
#         word_list.extend(list)
#     print(a,word_list)

In [98]:
text="t"
counter=6
skip_feature=[]
# ac.remove('pantie')
for i in ac:
#     print(i)
    cat_name=str(counter)
    counter+=1
    if i not in skip_feature:
        closest=glove_model.most_similar(i,topn=50)
        closest=[n for n,g in closest]
#         print(closest)
        closest_from_feature=[k for k in ac if k in closest and k!=i and k not in list_avoid]
#         print(i,closest_from_feature)
        if len(closest_from_feature)==0:
            category[cat_name]=[i]
        else:
            extended_list=[]
            for j in closest_from_feature:
#                 print("j",j)
                closest_j=glove_model.most_similar(j,topn=50)
                closest_j=[n for n,g in closest_j]
#                 print(closest_j)
                closest_from_feature_j=[k1 for k1 in ac if k1 in closest_j and k1!=i and k1 not in list_avoid]
#                 print(j,closest_from_feature_j)
                x=[x for x in closest_from_feature_j if x not in extended_list and x not in closest_from_feature]
                extended_list.extend(x)
#                 print("ex list",extended_list)
                skip_feature.extend(closest_from_feature_j)
            closest_from_feature.extend(extended_list)
            category[cat_name]=closest_from_feature
#             print("Final",closest_from_feature)
    else:
        pass

In [99]:
(category)

{'0': ['money',
  'buy',
  'price',
  'offer',
  'discount',
  'sale',
  'budget',
  'purchase',
  'penny',
  'payment',
  'pricing',
  'cost',
  'range',
  'mrp',
  'rs',
  'rupees'],
 '1': ['deliver', 'courier', 'delivery', 'guy'],
 '2': ['customer',
  'service',
  'flipkart',
  'company',
  'user',
  'consumer',
  'warranty',
  'experience'],
 '3': ['replacement',
  'refund',
  'cancel',
  'exchange',
  'replace',
  'policy',
  'return'],
 '4': ['package', 'shipment', 'packaging', 'packaged', 'box', 'packing'],
 '5': ['buyer', 'seller', 'distributor', 'market'],
 '6': ['quality', 'value', 'brand', 'level', 'work', 'image'],
 '7': ['response', 'week', 'complaint', 'action', 'today', 'mail'],
 '8': ['mobile'],
 '9': ['value', 'doubt', 'product', 'quality'],
 '10': ['video',
  'photo',
  'picture',
  'lens',
  'image',
  'watch',
  'recording',
  'quality',
  'thing'],
 '11': ['foot', 'side', 'hand', 'part', 'place', 'one', 'take'],
 '12': ['record', 'recording'],
 '13': ['cause', 'pro

In [240]:
#next step is to make our categories undestanding the various 

# category={'0': ['money',
#   'buy',
#   'price',
#   'offer',
#   'discount','amount','rupee',
#   'sale',
#   'budget',
#   'purchase',
#   'penny',
#   'payment',
#   'pricing',
#   'cost',
#   'range',
#   'mrp',
#   'rs',
#   'rupees'],
#  '1': ['deliver', 'courier', 'delivery',"address"],
#  '2': ['customer',
#   'service',
#   'flipkart',
#   'company',
#   'user',
#   'consumer',
#   'warranty',
#   'experience'],
#  '3': ['replacement',
#   'refund',
#   'cancel',
#   'exchange',
#   'replace',
#   'policy',
#   'return'],
#  '4': ['package', 'shipment', 'packaging', 'packaged', 'box', 'packing'],
#  '5': ['buyer', 'seller', 'distributor', 'market'],
#  '6':['description', 'detail','pic','photo','image','picture'],
#  '7': ['cotton', 'fabric', 'shirt', 'cloth','material','dress','clothing','silk','leather','denim'],
#  '8': ['colour', 'color'],
#  '9': [ 'design','stitching','embroidery','print','look'],
#  '10': ['size','fit','fitting','lenght','weight','waist']}

category =  {'0': ['money', 'buy', 'price', 'offer', 'discount', 'sale', 'budget', 'purchase', 'penny', 'payment', 'pricing', 'cost', 'range', 'mrp', 'rs', 'rupees'], '1': ['deliver', 'courier', 'delivery','guy'], '2': ['customer', 'service', 'flipkart', 'company', 'user', 'consumer', 'warranty', 'experience'], '3': ['replacement', 'refund', 'cancel', 'exchange', 'replace', 'policy', 'return'], '4': ['package', 'shipment', 'packaging', 'packaged', 'box', 'packing'], '5': ['buyer', 'seller', 'distributor', 'market'], 
'6': ['power','battery', 'charge', 'charger', 'light'], 
'7': ['cable','wire','plug','switch','pipe','cord'], 
'8': ['steel', 'aluminum','material','size', 'design'], 
'9': ['beard', 'shave', 'body','cut','oil','blade','razor','sharp','foam','speed'],
'10':['sound','noise'], 
'11':['hair','air','dry','dryer','speed','heat','temparature','blow'], 
'12':['weight','size','length','look'], 
"13":['fake','fraud'], 
'14':['brush','comb']}










In [241]:
print(len(data_lda))
data_check=[x for x in data_lda if len(x)!=0]

338


In [242]:
cat_number={}
for k,i in category.items():
    cat_number[k]=0
print(cat_number)

{'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0, '13': 0, '14': 0}


In [243]:
keywords_=[]
for i in data_lda :
    y=[k for k,j in category.items() if any(j1 in (i) for j1 in j )]
#     print(y,i)
    if y:
        for y1 in y:
            cat_number[y1]+=1
    else:
        y2=[(k1,j2,i1) for k1,j1 in category.items() for j2 in j1 for i1 in i if (fuzz.ratio(j2,i1)>80)]
        print(y2,i)
        if y2:
            
            for y21 in y2:
                print(y21[0])
                cat_number[y21[0]]+=1
        else:
            keywords_.append(i)

[] ['quality']
[] ['product']
[] []
[] ['product']
[] []
[] ['choice']
[] ['product']
[] []
[] ['machine']
[] ['photo']
[] ['system']
[] []
[] ['product']
[] ['quality', 'product']
[] ['expectation']
[] ['nightmare']
[] []
[] []
[] []
[] ['product']
[] ['product']
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] ['product']
[] []
[] []
[] ['product']
[] []
[] ['heating', 'shaving']
[] ['product']
[] []
[] ['product']
[] []
[] ['machine', 'scissor']
[] ['product']
[] []
[] []
[] []
[] ['product']
[] ['product']
[] []
[] []
[] ['product']
[] ['product']
[] ['quality', 'product']
[] ['product']
[] ['product']
[] ['stupidity']
[] ['woman']
[] []
[] []
[] ['product']
[] []
[] ['product']
[] ['quality']
[] []
[] []
[] ['product']
[] ['quality']
[] ['product']
[] ['product']
[] ['waste', 'product']
[] []
[] ['piece']
[] []
[] []
[] []
[] []
[] []
[] ['recommend', 'brand']
[] []
[] ['expectation']
[] ['product']
[] ['choice']
[] []
[] []
[] ['product']
[] ['product']
[] []
[] []
[] ['p

In [244]:
cat_number

{'0': 21,
 '1': 4,
 '2': 20,
 '3': 8,
 '4': 6,
 '5': 4,
 '6': 25,
 '7': 0,
 '8': 3,
 '9': 7,
 '10': 0,
 '11': 8,
 '12': 1,
 '13': 10,
 '14': 2}

In [245]:
print(len(data_lda))
print(len(keywords_))
print((len(keywords_)/len(data_lda))*100)

338
230
68.04733727810651


In [246]:
total=0
for n,l in cat_number.items():
    percent=(l/len(data_check))*100.0
    print(n,percent)
    total+=percent
print(total)

0 9.58904109589041
1 1.82648401826484
2 9.1324200913242
3 3.65296803652968
4 2.73972602739726
5 1.82648401826484
6 11.415525114155251
7 0.0
8 1.36986301369863
9 3.1963470319634704
10 0.0
11 3.65296803652968
12 0.45662100456621
13 4.5662100456621
14 0.91324200913242
54.33789954337898
